In [88]:
import math
import random
import copy

In [93]:
class individual:
    def __init__(self,code,startArray):
        self.codeWeight=1
        self.breakPointsWeight=10
        self.code=code
        self.array = []
        self.startArray = startArray
        self.applyReversals()
        self.breakpoints=self.getBreakpoints()
        self.fitness=self.calculateFitnes()
        
    def applyReversals(self):
        self.array = copy.deepcopy(self.startArray)
        for k in range(0,len(self.code),2):
            i = self.code[k]
            j = self.code[k+1]
            for l in range(int((j-i+1)/2)):
                self.array[i+l],self.array[j-l] = self.array[j-l],self.array[i+l]

    def getBreakpoints(self):
        breakPoints=0
        for i in range(1,len(self.array)):
            if abs(self.array[i-1]-self.array[i])>1:
                breakPoints+=1
        return breakPoints
    def calculateFitnes(self):
        return self.breakPointsWeight*self.breakpoints+self.codeWeight*len(self.code)

In [98]:
class MSR:
    def __init__(self,array):
        self.populationSize=500#math.floor(len(array)*math.log(len(array))) #Velicina populacije je floor(n*log n)
        self.array=array
        self.breakPoints=self.getBreakpoints()
        self.population=[]
        self.initializePopulation()
        self.tournamentSize=2#math.ceil(0.15*self.populationSize)
        self.bestUnit = copy.deepcopy(self.population[0])
        self.mutationRate = 0.01
        self.numOfIters = 1000000000
    def getBreakpoints(self):
        breakPoints=0
        for i in range(1,len(self.array)-1):
            if abs(self.array[i-1]-self.array[i])>1:
                breakPoints+=1
        return breakPoints
    def generateIndividual(self,size):
        code=[]
        for i in range(size):
            m=math.ceil(random.randrange(1,math.ceil(len(self.array)/10)+1))
            n=math.ceil(random.randrange(0,math.ceil(9*len(self.array)/10-1)))
            begin=n
            end=m+n
            code.append(begin)
            code.append(end)
        return individual(code,self.array)
            
    def initializePopulation(self):
        lowerBound=self.breakPoints
        upperBound=3*len(self.array)
        for i in range(self.populationSize):
            size=random.randrange(lowerBound,upperBound+1)
            self.population.append(self.generateIndividual(size))
    def selection(self):
        selected=random.sample(self.population,self.tournamentSize)
        best=min(selected,key=lambda x:x.fitness)
        #print("BEST: "+str(best.code))
        return best
    def crossover(self,parent1,parent2):
        if len(parent1)==len(parent2):
            while(True):
                firstPoint=random.randrange(0,len(parent1))
                secondPoint=random.randrange(0,len(parent1))
                if firstPoint!=secondPoint:
                    break
            if firstPoint>secondPoint:
                firstPoint,secondPoint=secondPoint,firstPoint
            child1=parent1[:firstPoint]+parent2[firstPoint:secondPoint]+parent1[secondPoint:]
            child2=parent2[:firstPoint]+parent1[firstPoint:secondPoint]+parent2[secondPoint:]
        else:
            if len(parent1)<len(parent2):
                parent1,parent2=parent2,parent1
            point=random.randrange(0,len(parent1)-len(parent2)+1)
            child1=parent1[:point]+parent2+parent1[point+len(parent2):]
            child2=parent1[point:point+len(parent2)]
        return individual(child1,self.array),individual(child2,self.array)

    def mutation1(self,unit):
        while(True):
            i = random.randrange(0,int(len(unit.code)/2))
            j = random.randrange(0,int(len(unit.code)/2))
            if i !=j and unit.code[i]!=unit.code[j]:
                break
        if j < i :
            i,j = j,i
        i = 2*i
        j = 2*j
        for k in range(j,len(unit.code)-1):
            unit.code[k] = unit.code[k+1]
        del unit.code[-1]
        for k in range(i,len(unit.code)-1):
            unit.code[k] = unit.code[k+1]
        del unit.code[-1]

        unit.applyReversals()
        unit.breakpoints = unit.getBreakpoints()
        unit.fitness = unit.calculateFitnes()
    def mutation2(self,unit):
        while(True):
            i = random.randrange(0,int(len(unit.code)/2))
            j = random.randrange(0,int(len(unit.code)/2))
            if i !=j:
                break
        if j < i :
            i,j = j,i
        i = 2*i
        j = 2*j
        while(True):
            value1=random.randrange(0,len(unit.array))
            value2=random.randrange(0,len(unit.array))
            if value1!=value2:
                break
        unit.code.insert(i,value1)
        unit.code.insert(j,value2)
        unit.applyReversals()
        unit.breakpoints = unit.getBreakpoints()
        unit.fitness = unit.calculateFitnes()

    def generatePopulation(self):
        population = []
        for i in range(0,self.populationSize,2):
            parent1 = self.selection()
            parent2 = self.selection()
            child1,child2 = self.crossover(parent1.code,parent2.code)
            if random.random() < self.mutationRate and len(child1.code)>2:
                self.mutation1(child1)
            if random.random() < self.mutationRate and len(child2.code)>2:
                self.mutation1(child2)
            if random.random() < self.mutationRate and len(child1.code)>2:
                self.mutation2(child1)
            if random.random() < self.mutationRate and len(child2.code)>2:
                self.mutation2(child2)
            population.append(child1)
            population.append(child2)
        self.population = copy.deepcopy(population)

    def findBest(self):
        best=min(self.population,key=lambda x:x.fitness)
        return copy.deepcopy(best)

    def solve(self):
        self.initializePopulation()
        for i in range(self.numOfIters):
            best = self.findBest()
            if best.fitness < self.bestUnit.fitness:
                self.bestUnit = copy.deepcopy(best)
            if i%10 == 0:
                #copyBest=copy.deepcopy(self.bestUnit)
                #copyBest.applyReversals()
                print("Najbolja jedinka:\nFitness: " + str(self.bestUnit.fitness) + "\nNiz: " + str(self.bestUnit.array)+"\nCode: "+str(self.bestUnit.code))
                print([self.population[i].code for i in range(20)])
                print()
            self.generatePopulation()
        #print("Gotovo")
        #print("Najbolja jedinka:\nFitness: " + str(self.bestUnit.fitness) + "\nBrojInverzija: " + str(len(self.bestUnit.code)/2))
        


In [100]:
msr=MSR([5,1,3,6,4,2,7])
msr.solve()
#t=msr.bestUnit
#t=individual([5, 6, 2, 3, 0, 2, 3, 0, 1, 1, 5, 6, 3, 4, 0, 1, 1, 5],[5,1,3,6,4,2,7])
#t.applyReversals()
#t.array

Najbolja jedinka:
Fitness: 32
Niz: [1, 5, 4, 3, 2, 6, 7]
Code: [0, 1, 3, 4, 2, 3, 4, 5, 1, 2, 1, 2]
[[3, 4, 4, 5, 5, 6, 3, 4, 0, 1, 1, 2, 0, 1, 2, 3, 1, 2, 5, 6, 3, 4, 5, 6, 0, 1, 2, 3, 5, 6, 2, 3, 4, 5, 2, 3, 2, 3, 2, 3], [4, 5, 2, 3, 2, 3, 5, 6, 5, 6, 5, 6, 3, 4, 0, 1, 4, 5, 4, 5], [5, 6, 3, 4, 5, 6, 1, 2, 1, 2, 1, 2, 1, 2, 0, 1, 0, 1, 3, 4, 0, 1, 1, 2, 0, 1, 5, 6, 0, 1, 5, 6, 3, 4], [3, 4, 2, 3, 5, 6, 3, 4, 4, 5], [5, 6, 0, 1, 1, 2, 2, 3, 4, 5, 3, 4, 4, 5, 3, 4, 0, 1, 5, 6, 1, 2, 4, 5, 4, 5, 4, 5, 3, 4, 3, 4, 0, 1, 3, 4, 5, 6, 5, 6], [2, 3, 4, 5, 0, 1, 2, 3, 2, 3, 4, 5, 2, 3, 2, 3], [2, 3, 3, 4, 4, 5, 1, 2, 3, 4, 2, 3, 1, 2, 2, 3, 0, 1, 5, 6], [1, 2, 1, 2, 0, 1, 1, 2, 5, 6, 2, 3, 3, 4, 5, 6, 5, 6, 3, 4, 1, 2, 5, 6, 2, 3, 0, 1, 5, 6], [0, 1, 0, 1, 5, 6, 1, 2, 3, 4, 1, 2, 3, 4, 4, 5, 3, 4, 1, 2, 4, 5, 5, 6, 3, 4, 4, 5, 1, 2], [1, 2, 3, 4, 5, 6, 2, 3, 4, 5, 2, 3, 3, 4, 1, 2, 2, 3, 5, 6, 1, 2, 1, 2, 5, 6, 2, 3, 1, 2], [4, 5, 3, 4, 1, 2, 1, 2, 2, 3, 5, 6, 4, 5, 0, 1, 3, 4, 2, 3], [3, 4, 

Najbolja jedinka:
Fitness: 8
Niz: [1, 2, 3, 4, 5, 6, 7]
Code: [0, 1, 3, 5, 1, 3, 3, 4]
[[3, 5, 1, 4, 4, 0, 2, 3], [3, 5, 1, 4, 4, 3, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 3, 2, 3], [3, 5, 1, 4, 4, 3, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 2, 2, 3], [3, 5, 1, 4, 4, 3, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 0, 2, 3], [3, 5, 1, 4, 4, 0, 2, 3], [3, 5, 1, 4, 6, 1, 2, 3], [3, 5, 1, 4, 4, 0, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 6, 0, 2, 3], [3, 5, 1, 4, 4, 4, 2, 3], [3, 5, 1, 4, 4, 3, 2, 3], [3, 5, 1, 4, 4, 2, 2, 3], [3, 5, 1, 4, 4, 0, 2, 3]]

Najbolja jedinka:
Fitness: 8
Niz: [1, 2, 3, 4, 5, 6, 7]
Code: [0, 1, 3, 5, 1, 3, 3, 4]
[[3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 0, 2, 3], [3, 5, 1, 4, 4, 0, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 3, 2, 3], [3, 5, 1, 4, 4, 2, 2, 3], [3, 5, 1, 4, 4, 0, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 0, 2, 3], [3, 5, 1, 4, 4, 0

Najbolja jedinka:
Fitness: 8
Niz: [1, 2, 3, 4, 5, 6, 7]
Code: [0, 1, 3, 5, 1, 3, 3, 4]
[[3, 5, 1, 4, 4, 2, 2, 3], [3, 5, 1, 4, 4, 3, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 2, 2, 3], [3, 5, 1, 4, 4, 3, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [5, 1, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 3, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 3, 2, 3], [3, 5, 1, 4, 4, 3, 2, 3], [3, 5, 1, 4, 4, 3, 2, 3], [3, 5, 1, 4, 4, 2, 2, 3], [3, 5, 1, 4, 4, 2, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3]]

Najbolja jedinka:
Fitness: 8
Niz: [1, 2, 3, 4, 5, 6, 7]
Code: [0, 1, 3, 5, 1, 3, 3, 4]
[[3, 5, 1, 4, 4, 2, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 3, 2, 3], [3, 5, 1, 4, 4, 2, 2, 3], [3, 5, 1, 4, 4, 2, 2, 3], [3, 5, 1, 4, 4, 3, 2, 3], [3, 5, 1, 4, 4, 3, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 3, 2, 3

Najbolja jedinka:
Fitness: 8
Niz: [1, 2, 3, 4, 5, 6, 7]
Code: [0, 1, 3, 5, 1, 3, 3, 4]
[[3, 5, 1, 4, 4, 2, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 2, 2, 3], [3, 5, 1, 4, 4, 3, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 2, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3]]

Najbolja jedinka:
Fitness: 8
Niz: [1, 2, 3, 4, 5, 6, 7]
Code: [0, 1, 3, 5, 1, 3, 3, 4]
[[3, 5, 1, 4, 4, 2, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 2, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 3, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 2, 2, 3], [3, 5, 1, 4, 4, 1

Najbolja jedinka:
Fitness: 8
Niz: [1, 2, 3, 4, 5, 6, 7]
Code: [0, 1, 3, 5, 1, 3, 3, 4]
[[3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 1, 4, 4, 6, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 2, 2, 3], [3, 5, 1, 4, 4, 2, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 2, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 2, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 2, 2, 3], [3, 5, 1, 4, 4, 2, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3]]

Najbolja jedinka:
Fitness: 8
Niz: [1, 2, 3, 4, 5, 6, 7]
Code: [0, 1, 3, 5, 1, 3, 3, 4]
[[3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 3, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 3, 2, 3], [3, 5, 1, 4, 4, 2, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 5, 4, 3, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 2, 2, 3], [3, 5, 1, 4, 4, 2, 2, 3], [3, 5

...

Najbolja jedinka:
Fitness: 8
Niz: [1, 2, 3, 4, 5, 6, 7]
Code: [0, 1, 3, 5, 1, 3, 3, 4]
[[3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3]]

Najbolja jedinka:
Fitness: 8
Niz: [1, 2, 3, 4, 5, 6, 7]
Code: [0, 1, 3, 5, 1, 3, 3, 4]
[[3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [5, 3, 1, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4

Najbolja jedinka:
Fitness: 8
Niz: [1, 2, 3, 4, 5, 6, 7]
Code: [0, 1, 3, 5, 1, 3, 3, 4]
[[3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3]]

Najbolja jedinka:
Fitness: 8
Niz: [1, 2, 3, 4, 5, 6, 7]
Code: [0, 1, 3, 5, 1, 3, 3, 4]
[[3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1

Najbolja jedinka:
Fitness: 8
Niz: [1, 2, 3, 4, 5, 6, 7]
Code: [0, 1, 3, 5, 1, 3, 3, 4]
[[3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [5, 1, 4, 4, 1, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3]]

Najbolja jedinka:
Fitness: 8
Niz: [1, 2, 3, 4, 5, 6, 7]
Code: [0, 1, 3, 5, 1, 3, 3, 4]
[[3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3

Najbolja jedinka:
Fitness: 8
Niz: [1, 2, 3, 4, 5, 6, 7]
Code: [0, 1, 3, 5, 1, 3, 3, 4]
[[3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3]]

Najbolja jedinka:
Fitness: 8
Niz: [1, 2, 3, 4, 5, 6, 7]
Code: [0, 1, 3, 5, 1, 3, 3, 4]
[[3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1, 2, 3], [3, 5, 1, 4, 4, 1

KeyboardInterrupt: 

In [ ]:
print(t.code)
print(t.array)
print(t.fitness)
#print(t.startArray)
#t.applyReversals()
#t.applyReversals()
print(t.fitness)
print(t.calculateFitnes())
print(t.breakpoints)
print(t.getBreakpoints())

In [ ]:
# msr.initializePopulation()
# for i in range(5):
#     best = msr.findBest()
#     if best.fitness < msr.bestUnit.fitness:
#         msr.bestUnit = best
#     if i %1== 0:
#         print("Najbolja jedinka:\nFitness: " + str(msr.bestUnit.fitness) + "\nNiz: " + str(msr.bestUnit.code))
#         [print(msr.population[i].code) for i in range(len(msr.population))]
#         print()
#     msr.generatePopulation()
#     print()
        

In [ ]:
t=individual([1, 3, 3, 0, 2, 3, 4, 5],[5,1,3,6,4,2,7])
print(t.array)
t.breakpoints

In [ ]:
msr.generateIndividual(4).code

In [ ]:
#num(breakpoint)+length(code)